<a href="https://colab.research.google.com/github/polivio/AM/blob/master/KDD_BR_Rel3l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/c/kddbr-2019/data

# KNN KDD BR -Polivio
import numpy as np
from sklearn import preprocessing, neighbors
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# auxílio para métricas
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics # https://youtu.be/6dbrR-WymjI

#modelos
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor


# Normalização
from sklearn.preprocessing import MinMaxScaler

In [0]:
# preprocessing # blob # raw
train = pd.read_csv('https://github.com/polivio/PEE/raw/master/Training_grouped.csv')
test = pd.read_csv('https://github.com/polivio/PEE/raw/master/Test_grouped.csv')



In [0]:
# separando o id, features e label para preprocessamento
idTrain = train.loc[:,['traininglabelorder.scatterplotID']]
features = train.loc[:,['qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette' ]]

labelTrain = train.loc[:,['traininglabelorder.score']]

features.head()

In [0]:
idVal = test.loc[:,['testorder.ID']]
featuresval = test.loc[:,[ 'qtde.ponto', 
       'mean.signalX', 'sd.signalX', 'kurtosis.signalX',
      'mean.signalY', 'sd.signalY', 
       'kurtosis.signalY', 'L', 'X', 'XY', 'Y',
       'mean.silhouette']]

#featuresval.head()

,qtde.ponto,mean.signalX,sd.signalX,kurtosis.signalX,mean.signalY,sd.signalY,kurtosis.signalY,L,X,XY,Y,mean.silhouette
0,384,2929.713542,983.267187,2.204854,2338.335938,880.798587,1.852330,7,116.0,167.0,94.0,0.916505
1,384,1409.484375,525.140811,2.798995,3155.841146,978.277806,2.624796,4,92.0,189.0,99.0,0.891220
2,382,2031.329843,2023.975889,2.531504,2982.876963,996.573219,2.506589,5,50.0,73.0,254.0,0.947689
3,370,2900.078378,1278.838337,1.794436,2463.178378,1937.785232,1.229172,5,170.0,10.0,185.0,0.954000
4,384,2567.598958,1090.155787,2.272346,3239.945312,1195.922708,2.315470,4,106.0,182.0,92.0,0.829513
5,384,4270.281250,2030.633675,2.377766,4901.104167,1866.213853,2.132134,7,32.0,221.0,124.0,0.419574
6,373,2978.777480,1572.195225,2.029055,1948.024129,1639.697019,2.898959,4,268.0,10.0,91.0,0.896077
7,384,757.755208,156.297509,5.168092,1348.734375,162.318779,6.400867,5,8.0,361.0,10.0,0.858830
8,112,1365.607143,945.558059,2.604198,3305.116071,1046.073790,3.044885,2,2.0,36.0,72.0,0.896741
9,212,1061.561321,563.184851,1.879929,1370.745283,293.410815,3.574951,3,25.0,64.0,120.0,0.953351


In [0]:

# change the dtype to 'float64' 
features = features.astype('float64')
featuresval = featuresval.astype('float64')

# Substituição pela média para os clusters com NA
features['X'].fillna(features['X'].mean(),inplace=True)
features['XY'].fillna(features['XY'].mean(),inplace=True)
features['Y'].fillna(features['XY'].mean(),inplace=True)



# Substituição pela média para os clusters com NA
featuresval['X'].fillna(featuresval['X'].mean(),inplace=True)
featuresval['XY'].fillna(featuresval['XY'].mean(),inplace=True)
featuresval['Y'].fillna(featuresval['XY'].mean(),inplace=True)


In [0]:
# Normalizando dados de treinamento = teste
print(scaler.fit(features))

features = scaler.transform(features)
print(features)


##  normalização antes
scaler = MinMaxScaler() 

# Normalizando dados de validação para serem usados no Kaggle
print(scaler.fit(featuresval))

featuresval = scaler.transform(featuresval)

print(featuresval)


MinMaxScaler(copy=True, feature_range=(0, 1))
[[1.         0.35069271 0.23304664 ... 0.43799472 0.24538259 0.95914333]
 [1.         0.1887101  0.1238514  ... 0.49604222 0.2585752  0.94168467]
 [0.99468085 0.25496863 0.48110143 ... 0.18997361 0.66754617 0.98067533]
 ...
 [0.99468085 0.4790201  0.57176292 ... 0.11609499 0.3298153  0.9492964 ]
 [1.         0.15521992 0.13268455 ... 0.02902375 0.49076517 0.98217308]
 [0.98670213 0.40946428 0.43198796 ... 0.55145119 0.2348285  0.94488508]]


In [0]:
# Separação de features e label para serem utilizadas em modelos
# separando o array em para 25% para teste com os dados de treinamento.

X_train, X_test, y_train, y_test = train_test_split(features, labelTrain, test_size=0.25, random_state=10)



Primeiro Modelo KNN

In [0]:
# Search for an optimal value of K for KNN
k_range = [1,3,5,7,11,13,17,19,23,29,31]
k_scores = []
for k in k_range:
  clf = neighbors.KNeighborsRegressor(n_neighbors=k)
  scores = cross_val_score(clf, X_train, y_train, cv = 10, scoring = 'neg_mean_squared_error') # não encontrei rmse direto
  k_scores.append(scores.mean())
  print(k)
print(k_scores)
  
  
  # Para k = 7 apresentou melhor valor 
  

1
3
5
7
11
13
17
19
23
29
31
[-0.08462996431326059, -0.060970033359571574, -0.05744442456110636, -0.056928566060806464, -0.05740667640186954, -0.05775505297510408, -0.05868360639566457, -0.05909398843847539, -0.05985399123543207, -0.0609149963117228, -0.06129042933101282]


In [0]:
# Cálculos para transformar na métrica em comum rmse
k_scores = [(x * -1 ) for x in k_scores]
print(k_scores)


[0.08462996431326059, 0.060970033359571574, 0.05744442456110636, 0.056928566060806464, 0.05740667640186954, 0.05775505297510408, 0.05868360639566457, 0.05909398843847539, 0.05985399123543207, 0.0609149963117228, 0.06129042933101282]
[-0.08462996431326059, -0.060970033359571574, -0.05744442456110636, -0.056928566060806464, -0.05740667640186954, -0.05775505297510408, -0.05868360639566457, -0.05909398843847539, -0.05985399123543207, -0.0609149963117228, -0.06129042933101282]


In [0]:
# raíz do menor erro

print(sqrt(0.056928566060806464))

0.23859707890250137


In [0]:
# Comando para validação de parâmetros de scorers.
#import sklearn 
#sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [0]:
# plot the value k for KNN (x-axis) versus the cross-validated accuracy (y-axis)
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('cross-validated accuracy')

In [0]:
# Comando para identricar os hiperparâmetros utilizados 
print(clf)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                    weights='uniform')


Old - testes anteriores sem cross validation e sem ver melhor número de n-neighbors, com base nos clusters (análise errada)
Dica de João Pedro sem número ímpar por desempate.

In [0]:
# Primeira versão 
# classsifiers KNN

clf = neighbors.KNeighborsRegressor(n_neighbors=4)
clf.fit(X_train , y_train)

# Validando com 25 % dos dados de treinametno.... E após fazer a validação com os dados de teste (20.000), atenção normalizar antes ...
y_pred = clf.predict(X_test)


In [0]:


accuracy = clf.score(X_test, y_test)
print(accuracy)
# ou

print metrics.accuracy_score(y_test, ypred)



0.48201400851719145


In [0]:
rmse = sqrt(mean_squared_error(y_pred, y_test))
print (rmse)

0.242298889493992


In [0]:
#Fazer testes no site do kaggle

Xval = featuresval
print(Xval)
# Estes dados associar ao Teste - ScatterPlotId  & este score e confirmar no site...
y_predVal = clf.predict(Xval)
print(y_predVal)

[[1.         0.35069271 0.23304664 ... 0.43799472 0.24538259 0.95914333]
 [1.         0.1887101  0.1238514  ... 0.49604222 0.2585752  0.94168467]
 [0.99468085 0.25496863 0.48110143 ... 0.18997361 0.66754617 0.98067533]
 ...
 [0.99468085 0.4790201  0.57176292 ... 0.11609499 0.3298153  0.9492964 ]
 [1.         0.15521992 0.13268455 ... 0.02902375 0.49076517 0.98217308]
 [0.98670213 0.40946428 0.43198796 ... 0.55145119 0.2348285  0.94488508]]
[[0.74665188]
 [0.78771325]
 [0.90561481]
 ...
 [0.77932236]
 [0.90898727]
 [0.34416327]]


Salvar o CSV e avaliar via Kaggle

In [0]:
# rever os comandos, primeiro necessário autenticação com drive e late submission.
from google.colab import files
files.download("dataset.csv")

FileNotFoundError: ignored

## Testes com Decision Tree Regressor

  

In [0]:
# Tuning de Hiperparâmetros da árvore
#Regressor



maxdepth = [3,5,7,11,13,17,19]
k_scores = []
for k in maxdepth:
  clf = tree.DecisionTreeRegressor(max_depth = k) # k = 11 (melhor predição)
  scores = cross_val_score(clf, X_train, y_train, cv = 10, scoring = 'neg_mean_squared_error') 
  k_scores.append(scores.mean())
  print(k)
print(k_scores)

# K = 11 menor valor

3
5
7
11
13
17
19
[-0.07406641120227511, -0.06545673238453882, -0.059307252791411136, -0.05502900820528511, -0.05785115610332621, -0.06919593864859323, -0.07337242930702262]


In [0]:
# somente o rmse com base no melhor parâmetro neg mean squared error
print(sqrt(0.05502900820528511))

0.23458262554009646


In [0]:
# Regressor

clf = tree.DecisionTreeRegressor(max_depth = 11)

clf.fit(X_train , y_train)

y_pred = clf.predict(X_test)

print(y_pred)

rmse = sqrt(mean_squared_error(y_pred, y_test))
print (rmse)

[0.77219937 0.83431929 0.14461588 ... 0.66893977 0.94592516 0.91619821]
0.23311026258173664


Testes com MLP Regressor

In [0]:
# https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html#sphx-glr-auto-examples-inspection-plot-partial-dependence-py
from sklearn.neural_network import MLPRegressor
nn=MLPRegressor(activation='relu',solver='sgd',hidden_layer_sizes=(10,15),random_state=1)

Testes com SVR

Testes com Ensemble